In [464]:
#Now we will prepare the dataset for pitchers. Pitchers have different
#stats so this analysis will require a separate dataframe and model.
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
mlb_salaries = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/mlbSalaries.csv')
p_stats = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/mlb2019statspitchers.csv')
p_stats.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,playerID,nameFirst,nameLast,throws,age,year,stints,teamID,LG,POS1,...,HRBIB,RAVG,FIP,DERA,STUFF,GURU,ROTO,mlbam_id,baseball_prospectus_id,n2019
0,abadfe01,Fernando,Abad,L,33,2019,1,SFN,NL,BP,...,0.07,4.15,4.51,4.56,15,10,$1.93,472551,ABAD19851217A,NaN
1,abreubr01,Bryan,Abreu,R,21,2019,1,HOU,AL,AP,...,-,1.04,1.24,1.31,465,"1,457",$2.42,650556,ABREU19970422A,4007.0
2,adamja01,Jason,Adam,R,27,2019,1,TOR,AL,BP,...,0.02,3.32,3.94,3.86,40,23,$5.09,592094,ADAM19910804A,4306.0
3,adamsau01,Austin,Adams,R,32,2019,2,DET,AL,BP,...,0.13,7.02,6.98,6.49,0,2,$-,542866,ADAMS19860819A,NaN
4,adamsau02,Austin,Adams,R,28,2019,2,SEA,AL,JP,...,0.10,3.94,3.11,3.04,387,65,$7.20,613534,ADAMS19910505A,1345.0
5,adamsch01,Chance,Adams,R,24,2019,1,NYA,AL,DP,...,0.15,8.88,6.52,5.91,8,4,$-,664856,ADAMS19940810A,138.0
6,adriaeh01,Ehire,Adrianza,R,29,2019,1,MIN,AL,AP,...,-,27.00,1.20,2.42,226,104,$-,501303,ADRIANZA19890821A,486.0
7,agrazda01,Dario,Agrazal,R,25,2019,1,PIT,NL,LP,...,0.09,5.28,5.89,5.71,0,11,$6.51,642607,AGRAZAL19941228A,5196.0
8,alanirj01,R.J.,Alaniz,R,28,2019,2,CIN,NL,BP,...,0.10,9.77,5.37,5.12,11,6,$-,595798,ALANIZ19910614A,4131.0
9,alberha01,Hanser,Alberto,R,26,2019,1,BAL,AL,AP,...,0.50,18.00,25.20,30.20,0,(0),$-,593643,ALBERTO19921017A,502.0


In [465]:
print(p_stats.columns)
#drop unwanted columns
dropped_cols = [0,5,44,45,46,47]
p_stats.drop(p_stats.columns[dropped_cols],axis=1,inplace=True)
p_stats.shape

Index(['playerID', 'nameFirst', 'nameLast', 'throws', 'age', 'year', 'stints',
       'teamID', 'LG', 'POS1', 'W', 'L', 'CG', 'ShO', 'GP', 'GS', 'SV', 'GF',
       'IPOuts', ' IP ', ' ERA ', 'HA', 'ER', 'HRA', 'BBA', 'SOA', 'IBBA',
       'WP', 'HPBA', 'BK', 'BFP', 'RA', ' CMD ', ' DOM ', ' CTL ', ' HR9 ',
       ' WHIP ', ' WRIP ', 'HRBIB', ' RAVG ', ' FIP ', ' DERA ', ' STUFF ',
       ' GURU ', ' ROTO ', ' mlbam_id ', ' baseball_prospectus_id ',
       ' n2019 '],
      dtype='object')


(832, 42)

In [466]:
p_stats.head(4)

,nameFirst,nameLast,throws,age,stints,teamID,LG,POS1,W,L,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
0,Fernando,Abad,L,33,1,SFN,NL,BP,0,2,...,2.1,1.4,0.92,1.77,0.07,4.15,4.51,4.56,15,10
1,Bryan,Abreu,R,21,1,HOU,AL,AP,0,0,...,3.1,-,0.81,0.35,-,1.04,1.24,1.31,465,"1,457"
2,Jason,Adam,R,27,1,TOR,AL,BP,3,0,...,4.2,0.4,1.15,0.92,0.02,3.32,3.94,3.86,40,23
3,Austin,Adams,R,32,2,DET,AL,BP,0,0,...,7.0,2.2,1.86,3.18,0.13,7.02,6.98,6.49,0,2


In [467]:
#put the name into one feature
for index, rows in p_stats.iterrows():
  p_stats.at[index,'nameFirst'] += ' ' + p_stats.at[index, 'nameLast']
p_stats.drop(p_stats.columns[1], axis = 1, inplace = True)
p_stats.rename(columns = {'nameFirst':'name'}, inplace = True)
p_stats.head(5)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
0,Fernando Abad,L,33,1,SFN,NL,BP,0,2,0,...,2.1,1.4,0.92,1.77,0.07,4.15,4.51,4.56,15,10
1,Bryan Abreu,R,21,1,HOU,AL,AP,0,0,0,...,3.1,-,0.81,0.35,-,1.04,1.24,1.31,465,"1,457"
2,Jason Adam,R,27,1,TOR,AL,BP,3,0,0,...,4.2,0.4,1.15,0.92,0.02,3.32,3.94,3.86,40,23
3,Austin Adams,R,32,2,DET,AL,BP,0,0,0,...,7.0,2.2,1.86,3.18,0.13,7.02,6.98,6.49,0,2
4,Austin Adams,R,28,2,SEA,AL,JP,2,2,0,...,4.5,1.1,1.13,1.75,0.10,3.94,3.11,3.04,387,65


In [468]:
#print the various values within POS1
#pitchers are traditionally split into Starting pitchers (SP) and
#relief pitchers (RP) so we will investigate what these labels refer to
print(p_stats['POS1'].value_counts())

LP    254
AP    175
JP    110
SP     83
BP     76
DP     71
EP     63
Name: POS1, dtype: int64


# We want to see what each of LP, AP, JP, SP, BP, DP, and EP refer to

# From looking at the innings pitched for lp's and from looking up players classified as LP's we can see that these are all relief pitchers

In [469]:
lp = p_stats.loc[p_stats['POS1'] == 'LP']
lp[' IP '].describe()

count    254.000000
mean      72.222441
std       27.903565
min       40.300000
25%       51.700000
50%       62.000000
75%       84.300000
max      163.300000
Name:  IP , dtype: float64

In [470]:
lp.head(5)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
7,Dario Agrazal,R,25,1,PIT,NL,LP,4,5,0,...,2.2,1.8,1.36,2.29,0.09,5.28,5.89,5.71,0,11
10,Matt Albers,R,36,1,MIL,NL,LP,8,6,0,...,4.4,1.2,1.37,1.83,0.07,5.13,4.64,4.40,43,22
13,Victor Alcantara,R,26,1,DET,AL,LP,3,2,0,...,3.2,1.7,1.41,2.23,0.08,5.27,5.71,5.64,0,8
15,Tyler Alexander,L,24,1,DET,AL,LP,1,4,0,...,1.2,1.5,1.40,1.81,0.09,5.03,4.13,3.98,66,30
16,Kolby Allard,L,21,1,TEX,AL,LP,4,2,0,...,3.8,0.6,1.57,1.08,0.03,5.16,3.99,4.04,20,26


# From looking at the spread of innings pitched, we see that AP's have pitched no more than 10 innings and by looking at the dataframe, this holds stats for many position players who had to pitch (but aren't really pitchers). We will drop these from the dataset.

In [471]:
ap = p_stats.loc[p_stats['POS1'] == 'AP']
ap[' IP '].describe()

count    175.000000
mean       4.165143
std        3.083472
min        0.000000
25%        1.300000
50%        3.300000
75%        6.700000
max       10.700000
Name:  IP , dtype: float64

In [472]:
ap.head(5)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
1,Bryan Abreu,R,21,1,HOU,AL,AP,0,0,0,...,3.1,-,0.81,0.35,-,1.04,1.24,1.31,465,"1,457"
6,Ehire Adrianza,R,29,1,MIN,AL,AP,0,0,0,...,-,-,5.00,-,-,27.00,1.20,2.42,226,104
9,Hanser Alberto,R,26,1,BAL,AL,AP,0,0,0,...,18.0,9.0,3.00,12.00,0.50,18.00,25.20,30.20,0,(0)
11,Jorge Alcala,R,23,1,MIN,AL,AP,0,0,0,...,5.4,-,1.20,0.60,-,-,3.80,4.01,3,11
21,Aaron Altherr,R,28,1,PHI,NL,AP,0,0,0,...,-,-,2.00,-,-,9.00,(0.80),0.52,"1,137","78,068"


# The JP's are relief pitchers as well.

In [473]:
jp = p_stats.loc[p_stats['POS1'] == 'JP']
jp[' IP '].describe()

count    110.000000
mean      21.390909
std       10.439274
min        0.300000
25%       13.300000
50%       21.850000
75%       29.600000
max       40.000000
Name:  IP , dtype: float64

In [474]:
jp.head(10)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
4,Austin Adams,R,28,2,SEA,AL,JP,2,2,0,...,4.5,1.1,1.13,1.75,0.10,3.94,3.11,3.04,387,65
18,Logan Allen,L,22,2,CLE,AL,JP,2,3,0,...,4.2,1.3,1.77,1.92,0.06,6.51,5.48,5.40,0,7
24,Adbert Alzolay,R,24,1,CHN,NL,JP,1,1,0,...,6.6,2.9,1.78,3.97,0.20,7.30,7.74,7.21,1,1
27,Cody Anderson,R,28,1,CLE,AL,JP,0,1,0,...,8.3,1.0,2.31,2.08,0.06,9.35,5.74,5.41,13,4
32,Tanner Anderson,R,26,1,OAK,AL,JP,0,3,0,...,2.8,1.6,1.66,2.10,0.09,6.45,4.86,4.45,23,13
33,Tyler Anderson,L,29,1,COL,NL,JP,0,3,0,...,4.8,3.5,2.13,4.40,0.26,11.76,7.60,6.64,11,2
38,Rogelio Armenteros,R,25,1,HOU,AL,JP,1,1,0,...,2.5,0.5,1.22,0.83,0.03,4.50,2.76,2.82,133,75
44,Pedro Avila,R,22,1,SDN,NL,JP,0,0,0,...,3.4,-,1.13,0.38,-,1.69,3.01,3.12,104,40
45,Michel Baez,R,22,1,SDN,NL,JP,1,1,0,...,4.2,0.9,1.31,1.48,0.05,3.03,4.35,4.08,52,21
53,Jacob Barnes,R,29,2,KCA,AL,JP,1,5,0,...,6.1,1.9,1.78,2.82,0.12,8.27,6.05,5.55,11,7


# The SP's refer to starting pitchers

In [475]:
sp = p_stats.loc[p_stats['POS1'] == 'SP']
sp[' IP '].describe()

count     83.000000
mean     175.459036
std       21.867835
min      130.700000
25%      161.500000
50%      174.700000
75%      192.700000
max      223.000000
Name:  IP , dtype: float64

In [476]:
sp.head(5)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
12,Sandy Alcantara,R,23,1,MIA,NL,SP,6,14,2,...,3.7,1.0,1.32,1.58,0.06,4.29,4.54,4.43,17,49
25,Brett Anderson,L,31,1,OAK,AL,SP,13,9,0,...,2.5,1.0,1.31,1.41,0.05,4.09,4.56,4.58,1,41
40,Jake Arrieta,R,33,1,PHI,NL,SP,8,8,0,...,3.4,1.4,1.47,1.92,0.08,5.04,4.87,4.66,20,32
48,Homer Bailey,R,33,2,OAK,AL,SP,13,9,0,...,2.9,1.2,1.32,1.61,0.07,4.63,4.09,3.98,58,57
63,Trevor Bauer,R,28,2,CIN,NL,SP,11,13,1,...,3.5,1.4,1.25,1.98,0.10,4.99,4.32,4.15,126,62


# From googling the players that we see in the dataframe below, we can see that BP also is comprised of relief pitchers

In [477]:
bp = p_stats.loc[p_stats['POS1'] == 'BP']
bp[' IP '].describe()

count    76.000000
mean     16.272368
std       3.220211
min      11.000000
25%      13.525000
50%      16.150000
75%      18.775000
max      21.700000
Name:  IP , dtype: float64

In [478]:
bp.head(10)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
0,Fernando Abad,L,33,1,SFN,NL,BP,0,2,0,...,2.1,1.4,0.92,1.77,0.07,4.15,4.51,4.56,15,10
2,Jason Adam,R,27,1,TOR,AL,BP,3,0,0,...,4.2,0.4,1.15,0.92,0.02,3.32,3.94,3.86,40,23
3,Austin Adams,R,32,2,DET,AL,BP,0,0,0,...,7.0,2.2,1.86,3.18,0.13,7.02,6.98,6.49,0,2
8,R.J. Alaniz,R,28,2,CIN,NL,BP,1,0,0,...,4.0,1.7,1.66,2.36,0.10,9.77,5.37,5.12,11,6
14,Scott Alexander,L,29,1,LAN,NL,BP,3,2,0,...,3.6,1.0,1.38,1.56,0.05,3.63,5.05,4.74,0,9
20,Dan Altavilla,R,26,1,SEA,AL,BP,2,1,0,...,7.4,0.6,1.43,1.50,0.04,5.52,4.09,3.91,94,19
65,David Bednar,R,24,1,SDN,NL,BP,0,2,0,...,4.1,2.5,1.36,3.18,0.19,6.55,5.56,5.19,92,5
69,Travis Bergen,L,26,1,SFN,NL,BP,2,0,0,...,4.1,1.8,1.37,2.49,0.11,5.49,5.54,5.09,21,7
76,Kyle Bird,L,26,1,TEX,AL,BP,0,0,0,...,10.7,3.6,2.05,5.13,0.22,7.82,10.78,10.90,0,0
79,Ray Black,R,29,2,MIL,NL,BP,0,1,0,...,5.1,2.8,1.44,3.69,0.20,5.06,6.89,6.61,19,2


# Looking at the players in DP, we see that these are also relief pitchers

In [479]:
dp = p_stats.loc[p_stats['POS1'] == 'DP']
dp[' IP '].describe()

count    71.000000
mean     30.174648
std       5.171839
min      22.000000
25%      26.000000
50%      29.700000
75%      33.850000
max      39.700000
Name:  IP , dtype: float64

In [480]:
dp.head(5)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
5,Chance Adams,R,24,1,NYA,AL,DP,1,1,0,...,3.9,2.5,1.97,3.20,0.15,8.88,6.52,5.91,8,4
17,Cody Allen,R,30,1,LAA,AL,DP,0,2,0,...,7.8,3.5,1.91,4.78,0.29,6.26,8.37,7.60,5,1
19,Yency Almonte,R,25,1,COL,NL,DP,0,1,0,...,3.7,1.9,1.56,2.47,0.11,5.82,5.49,5.17,14,9
43,Luis Avilan,L,29,1,NYN,NL,DP,4,0,0,...,3.9,1.4,1.47,2.00,0.08,5.06,4.95,4.84,36,11
56,Kyle Barraclough,R,29,2,SFN,NL,DP,1,2,0,...,5.6,2.4,1.75,3.30,0.17,6.42,6.35,5.64,41,6


# Looking up the players the EP's are also relief pitchers

In [481]:
ep = p_stats.loc[p_stats['POS1'] == 'EP']
ep[' IP '].describe()

count    63.000000
mean     65.406349
std      12.976913
min      28.700000
25%      59.850000
50%      67.300000
75%      72.700000
max      95.700000
Name:  IP , dtype: float64

In [482]:
ep.head()

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
23,Jose Alvarado,L,24,1,TBA,AL,EP,1,6,0,...,8.1,0.6,1.87,1.57,0.04,5.40,4.17,3.88,106,25
34,Matt Andriese,R,29,1,ARI,NL,EP,5,5,0,...,3.4,1.0,1.40,1.51,0.06,4.71,3.71,3.46,128,55
52,Scott Barlow,R,26,1,KCA,AL,EP,3,3,0,...,4.7,0.8,1.44,1.38,0.05,4.22,3.40,3.25,193,68
92,Archie Bradley,R,26,1,ARI,NL,EP,4,5,0,...,4.5,0.6,1.44,1.20,0.04,3.77,3.40,3.31,160,64
93,Ryan Brasier,R,31,1,BOS,AL,EP,2,4,0,...,3.4,1.5,1.29,1.99,0.09,5.34,4.40,4.17,94,26


# Now, we want to change those pitchers with position LP, JP, BP, DP, and EP to RP (relief pitcher), drop AP since it contains non-pitchers who pitched in games, and leave SP as is

In [483]:
for index, row in p_stats.iterrows():
  position = p_stats.at[index,'POS1']
  #drop the AP's
  if position == 'AP':
    p_stats.drop(labels = index, axis = 0, inplace = True)
  elif position in ['LP','JP','BP','DP','EP']:
    p_stats.at[index,'POS1'] = 'Reliever'
  elif position == 'SP':
    p_stats.at[index,'POS1'] = 'Starter'

In [484]:
p_stats.shape

(657, 41)

In [485]:
p_stats.head(20)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
0,Fernando Abad,L,33,1,SFN,NL,Reliever,0,2,0,...,2.1,1.4,0.92,1.77,0.07,4.15,4.51,4.56,15,10
2,Jason Adam,R,27,1,TOR,AL,Reliever,3,0,0,...,4.2,0.4,1.15,0.92,0.02,3.32,3.94,3.86,40,23
3,Austin Adams,R,32,2,DET,AL,Reliever,0,0,0,...,7.0,2.2,1.86,3.18,0.13,7.02,6.98,6.49,0,2
4,Austin Adams,R,28,2,SEA,AL,Reliever,2,2,0,...,4.5,1.1,1.13,1.75,0.10,3.94,3.11,3.04,387,65
5,Chance Adams,R,24,1,NYA,AL,Reliever,1,1,0,...,3.9,2.5,1.97,3.20,0.15,8.88,6.52,5.91,8,4
7,Dario Agrazal,R,25,1,PIT,NL,Reliever,4,5,0,...,2.2,1.8,1.36,2.29,0.09,5.28,5.89,5.71,0,11
8,R.J. Alaniz,R,28,2,CIN,NL,Reliever,1,0,0,...,4.0,1.7,1.66,2.36,0.10,9.77,5.37,5.12,11,6
10,Matt Albers,R,36,1,MIL,NL,Reliever,8,6,0,...,4.4,1.2,1.37,1.83,0.07,5.13,4.64,4.40,43,22
12,Sandy Alcantara,R,23,1,MIA,NL,Starter,6,14,2,...,3.7,1.0,1.32,1.58,0.06,4.29,4.54,4.43,17,49
13,Victor Alcantara,R,26,1,DET,AL,Reliever,3,2,0,...,3.2,1.7,1.41,2.23,0.08,5.27,5.71,5.64,0,8


In [486]:
#read in salaries dataframe and limit it to 2019 season
twentynineteen_salaries = mlb_salaries.loc[mlb_salaries['year'] == 2019]
twentynineteen_salaries.shape
twentynineteen_salaries.tail(5)

,year,team,name,salary,playerID
2283,2019,washington-nationals,Daniel Hudson,491965,543339
2284,2019,washington-nationals,Gerardo Parra,429696,467827
2285,2019,washington-nationals,Hunter Strickland,419355,519326
2286,2019,washington-nationals,Tanner Rainey,402840,663432
2287,2019,washington-nationals,Erick Fedde,355096,607200


In [487]:
twentynineteen_salaries = twentynineteen_salaries.drop('playerID', axis = 1)

In [488]:
#Before mapping names between the datasets we must get rid of any
#inconsistencies in the naming (remove Jr. and Sr. suffixes from both)
import re
for index,rows in p_stats.iterrows():
  if re.search('Jr.|Sr.',p_stats.at[index,'name']):
    p_stats.at[index,'name'] = p_stats.at[index,'name'][0:-4]
for index,rows in twentynineteen_salaries.iterrows():
  if re.search('Jr.|Sr.',twentynineteen_salaries.at[index,'name']):
    twentynineteen_salaries.at[index,'name'] = twentynineteen_salaries.at[index,'name'][0:-4]
#Alleviate one more issue with a last name inconsistency

#Alleviate first name issues: Write every player as the first two letters of their first name
#and then there last name to alleviate inconsistencies in nicknames between datasets
#For example some payers are referred to by nickname in war and full name in the 2019 stats dataframes
#This code will alleviate this issue and allow us to connect the two datasets via name
for index, rows in p_stats.iterrows():
  name_split = p_stats.at[index,'name'].split()
  if re.match('[A-Z]\.', name_split[1]):
    #print(name_split)
    name_split[1] = name_split[2]
  p_stats.at[index,'name'] = name_split[0][0:2] + ' ' + name_split[1]
  #print(war.at[index,'Name'])


for index, rows in twentynineteen_salaries.iterrows():
  name_split = twentynineteen_salaries.at[index,'name'].split()
  #If for some reason the entry has only one name for the player, delete the entry from dataframe
  if len(name_split) == 1:
    twentynineteen_salaries.drop(labels = index, axis = 0, inplace = True)
  else:
    if re.match('[A-Z]\.', name_split[1]):
      print(name_split)
      name_split[1] = name_split[2]
    twentynineteen_salaries.at[index,'name'] = name_split[0][0:2] + ' ' + name_split[1]
  #print(stats.at[index,'name'])

In [489]:
#alleivate issue where two players are both named austin adams
p_stats.at[3,'name'] = 'Aus Adams'
p_stats = p_stats.reset_index(drop = True)
p_stats.head()

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,CTL,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU
0,Fe Abad,L,33,1,SFN,NL,Reliever,0,2,0,...,2.1,1.4,0.92,1.77,0.07,4.15,4.51,4.56,15,10
1,Ja Adam,R,27,1,TOR,AL,Reliever,3,0,0,...,4.2,0.4,1.15,0.92,0.02,3.32,3.94,3.86,40,23
2,Aus Adams,R,32,2,DET,AL,Reliever,0,0,0,...,7.0,2.2,1.86,3.18,0.13,7.02,6.98,6.49,0,2
3,Au Adams,R,28,2,SEA,AL,Reliever,2,2,0,...,4.5,1.1,1.13,1.75,0.10,3.94,3.11,3.04,387,65
4,Ch Adams,R,24,1,NYA,AL,Reliever,1,1,0,...,3.9,2.5,1.97,3.20,0.15,8.88,6.52,5.91,8,4


In [490]:
#drop pitchers for which we do not have a 2019 salary
for index, row in p_stats.iterrows():
  if p_stats.at[index, 'name'] not in list(twentynineteen_salaries['name']):
    p_stats.drop(labels = index, axis = 0, inplace = True)
p_stats.shape

(377, 41)

In [491]:
player_salaries = {}
for index,row in twentynineteen_salaries.iterrows():
  player_salaries[twentynineteen_salaries.at[index,'name']] = twentynineteen_salaries.at[index,'salary']

# Add salary (in millions) onto the p_stats dataset

In [492]:
#add salary onto the pitcher stats dataframe
p_stats['salary'] = ''
for index, row in p_stats.iterrows():
  p_stats.at[index,'salary'] = player_salaries[p_stats.at[index,'name']]
#convert salary into millions
for index,row in p_stats.iterrows():
  p_stats.at[index,'salary'] = p_stats.at[index, 'salary']/1000000
p_stats.head(5)

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,HR9,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU,salary
0,Fe Abad,L,33,1,SFN,NL,Reliever,0,2,0,...,1.4,0.92,1.77,0.07,4.15,4.51,4.56,15,10,0.197846
3,Au Adams,R,28,2,SEA,AL,Reliever,2,2,0,...,1.1,1.13,1.75,0.10,3.94,3.11,3.04,387,65,0.41776
5,Da Agrazal,R,25,1,PIT,NL,Reliever,4,5,0,...,1.8,1.36,2.29,0.09,5.28,5.89,5.71,0,11,0.217832
7,Ma Albers,R,36,1,MIL,NL,Reliever,8,6,0,...,1.2,1.37,1.83,0.07,5.13,4.64,4.40,43,22,2.5
8,Sa Alcantara,R,23,1,MIA,NL,Starter,6,14,2,...,1.0,1.32,1.58,0.06,4.29,4.54,4.43,17,49,0.555


In [493]:
# want to add crucial feature of wins above replacement to our pitcher stats dataframe.
war = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/FanGraphs_Leaderboard.csv')

#Use regular expressions to remove suffixes from the names in WAR. 
#This will make it easier to get values from war into p_stats
#because names will be the same across the board
for index, rows in war.iterrows():
  if re.search('Jr.|Sr.',war.at[index,'Name']):
    war.at[index,'Name'] = war.at[index,'Name'][0:-4]

#Alleviate first name issues between p_stats and war: Write every player as the first two letters of their first name
#and then there last name to alleviate inconsistencies in nicknames between datasets
#For example some payers are referred to by nickname in war and full name in the 2019 stats dataframes
#This code will alleviate this issue and allow us to connect the two datasets via name
for index, rows in war.iterrows():
  name_split = war.at[index,'Name'].split()
  if re.match('[A-Z]\.', name_split[1]):
    name_split[1] = name_split[2]
  war.at[index,'Name'] = name_split[0][0:2] + ' ' + name_split[1]

In [494]:
players_war = {}
p_stats['WAR'] = ''
for index, rows in war.iterrows():
  players_war[war.at[index,'Name']] = war.at[index, 'Total WAR']
for index, rows in p_stats.iterrows():
  p_stats.at[index,'WAR'] = players_war[p_stats.at[index,'name']]
p_stats.shape

(377, 43)

In [495]:
p_stats.head()

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU,salary,WAR
0,Fe Abad,L,33,1,SFN,NL,Reliever,0,2,0,...,0.92,1.77,0.07,4.15,4.51,4.56,15,10,0.197846,-0.1
3,Au Adams,R,28,2,SEA,AL,Reliever,2,2,0,...,1.13,1.75,0.10,3.94,3.11,3.04,387,65,0.41776,-0.2
5,Da Agrazal,R,25,1,PIT,NL,Reliever,4,5,0,...,1.36,2.29,0.09,5.28,5.89,5.71,0,11,0.217832,-0.2
7,Ma Albers,R,36,1,MIL,NL,Reliever,8,6,0,...,1.37,1.83,0.07,5.13,4.64,4.40,43,22,2.5,0.0
8,Sa Alcantara,R,23,1,MIA,NL,Starter,6,14,2,...,1.32,1.58,0.06,4.29,4.54,4.43,17,49,0.555,2.1


In [496]:
#investigate null values and drop them
print(p_stats.isnull().sum())
p_stats = p_stats.dropna()

name       0
throws     0
age        0
stints     0
teamID     0
LG         0
POS1       0
W          0
L          0
CG         0
ShO        0
GP         0
GS         0
SV         0
GF         0
IPOuts     0
 IP        0
 ERA       0
HA         0
ER         0
HRA        0
BBA        0
SOA        0
IBBA       0
WP         0
HPBA       0
BK         0
BFP        0
RA         0
 CMD       2
 DOM       0
 CTL       0
 HR9       0
 WHIP      0
 WRIP      0
HRBIB      0
 RAVG      0
 FIP       0
 DERA      0
 STUFF     0
 GURU      0
salary     0
WAR        0
dtype: int64


In [497]:
#look at the distribution of pitcher stats
p_stats.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,375.0,28.394667,3.403988,19.00,26.00,28.00,31.00,39.00
stints,375.0,1.096000,0.303914,1.00,1.00,1.00,1.00,3.00
W,375.0,5.032000,4.314388,0.00,2.00,4.00,7.00,21.00
L,375.0,4.826667,3.612121,0.00,2.00,4.00,7.00,17.00
CG,375.0,0.098667,0.377684,0.00,0.00,0.00,0.00,3.00
ShO,375.0,0.056000,0.252390,0.00,0.00,0.00,0.00,2.00
GP,375.0,39.941333,18.744604,3.00,26.00,33.00,57.00,83.00
GS,375.0,10.146667,12.343451,0.00,0.00,2.00,20.50,34.00
SV,375.0,2.770667,7.387712,0.00,0.00,0.00,1.00,41.00
GF,375.0,8.845333,12.114100,0.00,0.00,5.00,11.00,56.00


In [498]:
p_stats.head()

,name,throws,age,stints,teamID,LG,POS1,W,L,CG,...,WHIP,WRIP,HRBIB,RAVG,FIP,DERA,STUFF,GURU,salary,WAR
0,Fe Abad,L,33,1,SFN,NL,Reliever,0,2,0,...,0.92,1.77,0.07,4.15,4.51,4.56,15,10,0.197846,-0.1
3,Au Adams,R,28,2,SEA,AL,Reliever,2,2,0,...,1.13,1.75,0.10,3.94,3.11,3.04,387,65,0.41776,-0.2
5,Da Agrazal,R,25,1,PIT,NL,Reliever,4,5,0,...,1.36,2.29,0.09,5.28,5.89,5.71,0,11,0.217832,-0.2
7,Ma Albers,R,36,1,MIL,NL,Reliever,8,6,0,...,1.37,1.83,0.07,5.13,4.64,4.40,43,22,2.5,0.0
8,Sa Alcantara,R,23,1,MIA,NL,Starter,6,14,2,...,1.32,1.58,0.06,4.29,4.54,4.43,17,49,0.555,2.1


In [499]:
p_stats.to_csv('pitcher_stats_clean.csv', index = False)

In [500]:
import sklearn
from sklearn.model_selection import train_test_split

# Utilize a 80-20 train test split of the data (we will use for our model building)

In [501]:
pitcher_train, pitcher_test = train_test_split(p_stats, test_size = 0.2, random_state = 5)

In [502]:
print('Train:')
print('Train dataset dimensions: ', pitcher_train.shape, '\n')

print('Test:')
print('Test dataset dimensions: ', pitcher_test.shape, '\n')

Train:
Train dataset dimensions:  (300, 43) 

Test:
Test dataset dimensions:  (75, 43) 



In [503]:
pitcher_train.to_csv('pitcher_train.csv', index = False)
pitcher_test.to_csv('pitcher_test.csv', index = False)

# In the event that models struggle to predict pitcher salary, I may try to predict reliever salary and starting pitcher salary with their own models. So I will get training and testing data for this too now

In [504]:
starters = p_stats.loc[p_stats['POS1'] == 'Starter']
starters.shape

(72, 43)

In [505]:
relievers = p_stats.loc[p_stats['POS1'] == 'Reliever']
relievers.shape

(303, 43)

In [506]:
starters_train, starters_test = train_test_split(starters, test_size = 0.2, random_state = 5)

In [507]:
print('Train:')
print('Train dataset dimensions: ', starters_train.shape, '\n')

print('Test:')
print('Test dataset dimensions: ', starters_test.shape, '\n')

Train:
Train dataset dimensions:  (57, 43) 

Test:
Test dataset dimensions:  (15, 43) 



In [508]:
relievers_train, relievers_test = train_test_split(relievers, test_size = 0.2, random_state = 5)

In [509]:
print('Train:')
print('Train dataset dimensions: ', relievers_train.shape, '\n')

print('Test:')
print('Test dataset dimensions: ', relievers_test.shape, '\n')

Train:
Train dataset dimensions:  (242, 43) 

Test:
Test dataset dimensions:  (61, 43) 



In [511]:
relievers_train.to_csv('relievers_train.csv', index = False)
relievers_test.to_csv('relivers_test.csv', index = False)
starters_train.to_csv('starters_train.csv', index = False)
starters_test.to_csv('starters_test.csv', index = False)